In [1]:
#Exercise 1 (part 1)

def read_csv_file(input):
    '''this function uses dictreader to create an object that operates like a regular reader but maps the information in each row to a dictionary; the for loop adds the data from the csv file to an empty list - to be used later in calculations'''
    file_path = input("Enter the path to the CSV file: ") 
    records = [] 
    with open(file_path, 'r') as file:
        csv_reader = csv.DictReader(file)
        for row in csv_reader:
            records.append(row)

In [2]:
#Exercise 1 (part 2)

def average_grade_calculator(records):
    '''this function takes as input the records that were read earlier: it sums the (transformed float) data from the records list in the grade column, and divide by the length of the list to then print the average grade'''
    total = sum(float(record['Grade']) for record in records)
    average = total / len(records)
    return average

In [3]:
#Exercise 1 (part 3)

def filtering_function(records):
    '''this function uses the entire row in the records list to filter students who got a grade above 80, and then prints for each 'filtered' entry the name and grade'''
    filtered_records = [record for record in records if float(record['Grade']) >= 80.0]
    print("Student Report")
    print("--------------")
    for record in filtered_records:
        print(f"Name: {record['Name']}")
        print(f"Grade: {record['Grade']}")
        print("--------------------")

In [ ]:
#Exercise 2 (part 1)

def a(order, c):
    '''this function generates the number that is asked for in the input: the 'order'th mandelbrot number with the constant c specified'''
    if order==0:
        print('a_0=0')
    else:
        return a(order-1)**2+c

